In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Folder containing the images
image_folder = '/Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360pics'
output_folder = '/Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360picsthermal'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Keypoint vulnerability scores (example)
keypoint_scores = {
    'NOSE': 0.9,
    'LEFT_SHOULDER': 0.8,
    'RIGHT_SHOULDER': 0.8,
    'LEFT_ELBOW': 0.6,
    'RIGHT_ELBOW': 0.6,
    'LEFT_HIP': 0.7,
    'RIGHT_HIP': 0.7,
    'LEFT_WRIST': 0.5,
    'RIGHT_WRIST': 0.5,
    'LEFT_ANKLE': 0.3,
    'RIGHT_ANKLE': 0.3
}

# Process all images in the folder
for image_name in os.listdir(image_folder):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, image_name)
        print(f"Processing image: {image_path}")
        
        # Load the image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detect pose
        results = pose.process(image_rgb)
        
        # Heatmap canvas
        heatmap = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)
        
        # Draw 2D Gaussians at keypoints
        if results.pose_landmarks:
            h, w = image.shape[:2]
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                landmark_name = mp_pose.PoseLandmark(idx).name
                if landmark_name in keypoint_scores:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    score = keypoint_scores[landmark_name]
                    
                    # Draw Gaussian on heatmap
                    radius = 50
                    y_grid, x_grid = np.ogrid[-y:h-y, -x:w-x]
                    mask = np.exp(-(x_grid**2 + y_grid**2) / (2 * (radius**2))) * score
                    heatmap = np.maximum(heatmap, mask)
        
        # Normalize heatmap
        heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Apply color map
        heatmap_color = cv2.applyColorMap(heatmap_norm, cv2.COLORMAP_JET)
        
        # Blend heatmap with original image
        blended = cv2.addWeighted(image, 0.7, heatmap_color, 0.5, 0)
        
        # Save the result to the new folder
        output_path = os.path.join(output_folder, f"thermal_{image_name}")
        cv2.imwrite(output_path, blended)
        print(f"Saved processed image to: {output_path}")

print("Processing complete.")


I0000 00:00:1745174503.310349 4913221 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745174503.404980 4913337 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745174503.417015 4913339 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745174503.431735 4913333 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processing image: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360pics/63.jpg
Saved processed image to: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360picsthermal/thermal_63.jpg
Processing image: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360pics/77.jpg
Saved processed image to: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360picsthermal/thermal_77.jpg
Processing image: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360pics/88.jpg
Saved processed image to: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360picsthermal/thermal_88.jpg
Processing image: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360pics/89.jpg
Saved processed image to: /Users/hridayunadkat/Desktop/Technical Projects/dronehack/dronehackathon/360picsthermal/thermal_89.jpg
Processing image: /Users/hridayunadkat/Desktop/Technical Pro